# Investigation 002: High-Res Stability Mapping  

**File:** 002-high-res-stability-map.ipynb  
**Date Started:** 2025-06-08  
**Date Updated:** 2025-06-09  


## Introduction

The previous investigation discovered eight stable field configurations by scanning the central field parameter `C(0)` from 0.1 to 5.0 in 50 steps, with all eight falling within range 0.1-0.8. The goal now is to look much more carefully at that region using 1000 steps—going from `C(0) = 0.001` to `1.0` with higher resolution.

Those eight configurations are likely not the whole story, and the scan may have missed something important by looking too coarsely. Maybe they're part of larger families of solutions. Maybe there are more configurations hiding between the points checked. Maybe some of what was found were just numerical artifacts.

The first goal is to find every stable solution in this parameter range. Second, figure out exactly where stable solutions start and stop existing (within 0.001 accuracy). Third, check if the masses of these solutions follow any regular mathematical pattern. Fourth, map out any "forbidden zones" where no stable solutions can exist.

Success will be measured by (1) how completely the solution space is mapped, (2) how precisely the stability thresholds are identified, (3) whether mass patterns hold up statistically, and (4) whether results cross-validate with those of *001-stable-soliton*. The scope remains spherically symmetric and static (no time evolution or higher-dimensional topologies yet), but the parameter range is much finer and more targeted than before.

## Investigation

**Question:**  
What structure of stable field configurations emerges when RFT field equations are solved at high resolution across the full parameter space?

**Working Hypothesis:**  
The eight stable configurations found in *001-stable-soliton* might just be the tip of the iceberg. With 10× better resolution and a narrower range, the expectation is to discover whether these are isolated solutions or part of continuous families, and whether there are precise mathematical patterns governing when stable solitons can exist.

**Approach:**  
Systematically solve the field equations at 1000 points from C(0) = 0.001 to C(0) = 1.0, paying special attention to boundary regions where solutions appear or disappear, and looking for patterns in the masses and sizes of stable configurations.

**What is sought:**
1. **Complete Census**: Find and document every stable field configuration in the parameter range
2. **Sharp Boundaries**: Pin down exactly where stable solutions start and stop existing (within ±0.001 accuracy)
3. **Mass Patterns**: Test whether the masses follow regular mathematical relationships
4. **Solution Families**: Determine if solutions come in discrete types or continuous families
5. **Forbidden Zones**: Identify parameter ranges where no stable solutions exist

**Success Means:**
- **Complete Mapping**: Finding stable solutions for all values of C(0) in the parameter range
- **Precise Boundaries**: Knowing exactly where stability transitions occur
- **Pattern Validation**: Confirming or refuting the mass quantization hypothesis with statistical confidence
- **Consistency Check**: Results matching *001-stable-soliton* in the overlapping regions

**Investigation Scope:**
- **Parameter Range**: C(0) from 0.001 to 1.0 (covering 3 orders of magnitude)
- **Resolution**: 1000 points (vs. 50 in the previous investigation)
- **Geometry**: Spherically symmetric solutions only
- **Time**: Static solutions only (no time evolution)

This high-resolution scan should reveal more of the mathematical structure underlying stable soliton formation in RFT.


## Mathematical Context

This extends the computational framework from *001-stable-soliton*, using the same core **Recurgent Field Equation**:

$$\Box C_i = T^{\text{rec}}_{ij} \cdot g^{jk} C_k$$

For spherically symmetric solutions, this simplifies to:

$$\frac{d^2 C}{dr^2} + \frac{2}{r}\frac{dC}{dr} = F(C, R, \rho)$$

where $F$ captures how the coherence field $C$ couples with recursive structure $R$ and constraint density $\rho$.

### Semantic Mass Framework

Measure the "semantic mass" of each stable configuration:

$$M_{\text{semantic}} = \int_V D(r) \cdot \rho(r) \cdot A(r) \, d^3r$$

This combines three physical aspects:
- **$D(r)$ — Recursive Depth**: How much the field resists recursive feedback
- **$\rho(r)$ — Constraint Density**: How tightly the field is geometrically constrained
- **$A(r)$ — Attractor Stability**: How strongly the field returns to equilibrium after perturbation

### Statistical Analysis Tools

**Solution Density:**
$$\rho_{\text{sol}}(C_0) = \frac{N_{\text{stable}}(C_0, C_0 + \Delta C_0)}{\Delta C_0}$$

**Mass Ratios:**
$$r_{n} = \frac{M_{n+1}}{M_n} \quad \text{with uncertainty } \delta r_n$$

**Scaling Laws:**
$$L \propto M^{\alpha} \quad \text{with fitted exponent: } \alpha \pm \delta\alpha$$

These tools will reveal whether solutions form discrete families (with sharp mass ratios) or continuous spectra (with smooth scaling relationships).


In [ ]:
# This section loads the mathematical tools needed to solve the field equations.
 
# It also sets some basic rules for the overall investigation:
# - Searches for solutions in a specific range (from 0.001 to 1.0)
# - Uses very high precision (accurate to 9 decimal places)
# - Uses the same starting point each time, so the results are reproducible

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, quad
from scipy.optimize import minimize, root_scalar
from scipy.stats import linregress
import warnings
warnings.filterwarnings('ignore')

# Investigation parameters
np.random.seed(42)                   # Consistent state for reproducibility
RANGE_START, RANGE_END = 0.001, 1.0  # Parameter search range (extended to capture 001's solutions)
TOLERANCE = 1e-9                     # Field value is considered zero if it's less than 0.000000001

print(f"Script initialized.")
print(f"Seed has been set to {np.random.get_state()[1][0]}.")


In [ ]:
# This defines the field equation as a standalone function, so it can be called later by the solver.

def rft_field_equation(r, C, alpha=1.0, beta=1.0):
    """
    Spherically symmetric RFT field equation for soliton solutions.
    
    d²C/dr² + (2/r)dC/dr = F(C, R, ρ)
    
    Starting with nonlinear coupling: F(C) = -α*C + β*C³
    This produces natural stability without external scale setting.
    
    Args:
        r: radial coordinate
        C: [C_val, dC_dr] field value and derivative
        alpha, beta: coupling parameters
    
    Returns:
        [dC_dr, d²C_dr²] derivatives for integration
    """
    # Unpack the field state vector
    C_val, dC_dr = C
    
    # Natural nonlinear coupling (no external scale setting)
    # -α*C provides restoring force, +β*C³ provides nonlinear stabilization
    nonlinear_term = -alpha * C_val + beta * C_val**3
    
    # Second derivative from spherically symmetric field equation
    # Handle r=0 singularity by setting 2/r term to zero for small r
    spherical_term = (2/r if r > 1e-10 else 0) * dC_dr
    d2C_dr2 = nonlinear_term - spherical_term
    
    # Return derivatives for scipy.integrate.solve_ivp
    return [dC_dr, d2C_dr2]

print("Field equation set.")

In [ ]:
# This generates the full 1000-point parameter grid to work with
N_POINTS = 1000
central_values_full = np.linspace(RANGE_START, RANGE_END, N_POINTS)

# Now to calculate key statistics for comparison with 001-stable-soliton
overlap_mask = (central_values_full >= 0.1) & (central_values_full <= 1.0)
new_region_mask = central_values_full < 0.1

print("Parameter Grid Configuration:")
print(f"  Range to test: C(0) ∈ [{RANGE_START:.3f}, {RANGE_END:.3f}]")
print(f"  Total points: {N_POINTS}")
print()
print("Comparison with 001-stable-soliton:")
print(f"  Previous study range: [0.1, 5.0] with 50 points")
print(f"  Current study range: [0.001, 1.0] with 1000 points")
print()
print(f"  Overlap region [0.1, 1.0]: {np.sum(overlap_mask)} points (current)")
print(f"  Overlap region [0.1, 1.0]: 10 points (previous)")
print(f"  Resolution improvement in overlap: {np.sum(overlap_mask)/10:.1f}×")
print()
print("New parameter space exploration:")
print(f"  Low-field region [0.001, 0.1): {np.sum(new_region_mask)} points")
print(f"  This explores previously uncharted territory in weak-field regimes")


## Radial Grid Setup: Defining the "space" where the field lives

Since this analysis studies a field, imagine it like the sphere of heat and light around a campfire. That field has properties that can be described by specific values at every point in the space around it. In this case, the focus is on the "coherence field" `C(r)`, which describes how the field changes as distance increases from the center.

The field equation above tells how this field behaves. The expectation is that it will be strongest at the center, and then gradually fade to nothing at greater distances.

But in this case, exactly how it fades, or even if it will fade at all for every starting condition, remains unknown. That's what this investigation hopes to determine.

To study this, the analysis must decide where to look and how carefully to look. It's like deciding where to place thermometers around that campfire. Put them too close together and you waste all your effort measuring nearly identical temperatures. Put them too far apart and important changes get missed.

The smart approach is to place more measurement points where things change rapidly (close to the center) and fewer points where things change slowly (far from the center). This is exactly what the "radial grid" accomplishes - the set of distances where the field value will be calculated.

In [ ]:

# Use logarithmic spacing to efficiently sample across these different scales
R_MIN = 1e-3    # Start very close to center (but not exactly zero to avoid singularity)
R_MAX = 100.0   # Go far enough to see complete decay
N_RADIAL = 1000 # Enough points to capture smooth field profiles

r_points = np.logspace(np.log10(R_MIN), np.log10(R_MAX), N_RADIAL)

# Why logarithmic spacing?
# Linear spacing: [0.001, 0.002, 0.003, ..., 99.998, 99.999, 100.0]
# - Wastes most points in the boring region where field ≈ 0
# - Misses fine structure near the center where field changes rapidly

# Logarithmic spacing: [0.001, 0.0011, 0.0012, ..., 91.2, 100.0]  
# - Dense sampling where field changes rapidly (small r)
# - Sparse sampling where field is nearly constant (large r)
# - Captures the natural scales of the physics

print("Radial Grid Configuration:")
print(f"  Range: r ∈ [{R_MIN:.3f}, {R_MAX:.1f}]")
print(f"  Points: {N_RADIAL}")
print(f"  Spacing: Logarithmic (dense near center, sparse at large r)")
print()
print("Scale coverage:")
print(f"  Near-field (r < 1): {np.sum(r_points < 1)} points")
print(f"  Mid-field (1 ≤ r < 10): {np.sum((r_points >= 1) & (r_points < 10))} points") 
print(f"  Far-field (r ≥ 10): {np.sum(r_points >= 10)} points")
print()
print("Why this matters:")
print("  - Near-field: Captures the 'core' of each soliton")
print("  - Mid-field: Maps the main body and characteristic size")
print("  - Far-field: Verifies the field actually decays to zero")
print("  - Logarithmic spacing: Efficient sampling across all relevant scales")


## Solution Parameter Sweep

This will systematically solve the field equation for each of the 1000 central field values, test each solution against stability criteria, and collect all the stable configurations.

The process:
1. **For each C(0) value** from 0.001 to 1.0 (1000 total)
2. **Solve the field equation** from center outward to large distances
3. **Test stability criteria**: Does it decay to zero? Stay finite? No pathological behavior?
4. **If stable**: Save the complete solution data
5. **Track progress**: Monitor how many solutions are found in different parameter regions

This will reveal whether the 8 solutions from 001 are isolated points or part of continuous families, and whether there are additional solutions in the previously unexplored low-field region [0.001, 0.1).


In [ ]:
# Main solution discovery engine
# This systematically searches for stable solitons across the full parameter range

def find_all_stable_solutions(central_values, r_points, tolerance=1e-9):
    """
    Comprehensive search for stable field configurations across parameter space.
    
    Args:
        central_values: Array of C(0) values to test
        r_points: Radial grid for field integration  
        tolerance: Convergence tolerance for field equations
        
    Returns:
        List of stable solution dictionaries with complete field data
    """
    stable_solutions = []
    failed_integrations = 0
    
    print(f"Systematic Parameter Sweep: Testing {len(central_values)} field configurations...")
    print(f"Parameter range: C(0) ∈ [{central_values[0]:.3f}, {central_values[-1]:.3f}]")
    print(f"Integration tolerance: {tolerance:.0e}")
    print()
    
    # Progress tracking
    checkpoint_interval = len(central_values) // 10  # Report every 10%
    
    for i, C_center in enumerate(central_values):
        try:
            # Initial conditions for spherically symmetric solution
            # C(0) = C_center (specified central field value)
            # dC/dr(0) = 0 (spherical symmetry requires zero derivative at center)
            initial_conditions = [C_center, 0.0]
            
            # Solve the RFT field equation (match Investigation 001's integration settings)
            solution = solve_ivp(
                rft_field_equation,
                [r_points[0], r_points[-1]], 
                initial_conditions,
                t_eval=r_points,
                method='RK45',
                rtol=1e-8  # Match 001's tolerance exactly
            )
            
            if solution.success:
                C_r = solution.y[0]  # Field values C(r)
                dC_dr = solution.y[1]  # Field derivatives dC/dr(r)
                
                # Apply stability criteria EXACTLY as in Investigation 001
                # (Remove extra constraints that were filtering out valid solutions)
                
                # Criterion 1: Field decays to negligible value at large distances
                asymptotic_value = abs(C_r[-10:].mean())  # Match 001: last 10 points
                
                # Criterion 2: Field remains finite everywhere (no singularities)  
                is_finite = np.all(np.isfinite(C_r))  # Match 001: only check C_r
                
                # Criterion 3: Field localizes (decays to zero)
                is_localized = asymptotic_value < 0.001  # Match 001: same threshold
                
                # Accept as stable soliton using 001's EXACT criteria
                # (Remove the problematic 'is_nontrivial' constraint)
                if is_finite and is_localized:
                    stable_solutions.append({
                        'C_center': C_center,
                        'r': r_points.copy(),
                        'C': C_r.copy(),
                        'dC_dr': dC_dr.copy(),
                        'asymptotic_value': asymptotic_value,
                        'parameter_index': i
                    })
            else:
                failed_integrations += 1
                
        except Exception as e:
            failed_integrations += 1
            continue
        
        # Progress reporting
        if (i + 1) % checkpoint_interval == 0 or i == len(central_values) - 1:
            progress = 100 * (i + 1) / len(central_values)
            current_solutions = len(stable_solutions)
            print(f"Progress: {progress:5.1f}% | Stable solutions found: {current_solutions:3d} | Failed integrations: {failed_integrations:3d}")
    
    print(f"\nSearch complete!")
    print(f"Total stable solutions discovered: {len(stable_solutions)}")
    print(f"Failed integrations: {failed_integrations}")
    print(f"Success rate: {100 * (len(central_values) - failed_integrations) / len(central_values):.1f}%")
    
    return stable_solutions

# Execute the systematic parameter sweep
print("=" * 80)
print("HIGH-RESOLUTION STABILITY MAPPING")
print("Investigation 002: Complete Parameter Space Survey")
print("=" * 80)
print()

all_stable_solutions = find_all_stable_solutions(central_values_full, r_points, tolerance=TOLERANCE)


## Solution Analysis

Now that the systematic sweep is complete, the discovered solutions can be analyzed to understand the complete stability landscape. The high-resolution results will be compared with *001-stable-soliton* to determine how many of the original 8 solutions found there are also present in the high-resolution scan, map where stable solutions exist throughout parameter space, identify any new configurations that were missed by the coarser search, and characterize the precise boundaries where stable solutions begin and cease to exist. This comprehensive analysis will provide the complete picture of the stability landscape that *001-stable-soliton* could only partially reveal.


In [ ]:
# Comprehensive analysis of discovered solutions

if len(all_stable_solutions) > 0:
    # Extract key properties for analysis
    central_vals = np.array([sol['C_center'] for sol in all_stable_solutions])
    asymptotic_vals = np.array([sol['asymptotic_value'] for sol in all_stable_solutions])
    parameter_indices = np.array([sol['parameter_index'] for sol in all_stable_solutions])
    
    print("SOLUTION LANDSCAPE ANALYSIS")
    print("=" * 50)
    print(f"Total stable configurations found: {len(all_stable_solutions)}")
    print(f"Parameter range with solutions: C(0) ∈ [{central_vals.min():.4f}, {central_vals.max():.4f}]")
    print(f"Solution density: {len(all_stable_solutions)/len(central_values_full):.1%} of parameter space")
    print()
    
    # Compare with Investigation 001 results
    print("COMPARISON WITH INVESTIGATION 001:")
    print("-" * 35)
    
    # Investigation 001 found solutions at C(0) = 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8
    investigation_001_values = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
    
    confirmed_solutions = 0
    for val_001 in investigation_001_values:
        # Check for a solution within ±0.005 of each 001 solution
        nearby_solutions = np.abs(central_vals - val_001) < 0.005
        if np.any(nearby_solutions):
            confirmed_solutions += 1
            closest_idx = np.argmin(np.abs(central_vals - val_001))
            closest_val = central_vals[closest_idx]
            print(f"  ✓ C(0) = {val_001:.1f}: Confirmed at C(0) = {closest_val:.4f}")
        else:
            print(f"  ✗ C(0) = {val_001:.1f}: Not found in high-resolution scan")
    
    print(f"\nValidation: {confirmed_solutions}/{len(investigation_001_values)} solutions from 001 confirmed")
    print(f"New discoveries: {len(all_stable_solutions) - confirmed_solutions} additional stable configurations")
    print()
    
    # Solution distribution analysis
    print("PARAMETER SPACE DISTRIBUTION:")
    print("-" * 30)
    
    # Divide parameter space into regions for analysis
    low_field_mask = central_vals < 0.1
    mid_field_mask = (central_vals >= 0.1) & (central_vals < 0.5)
    high_field_mask = central_vals >= 0.5
    
    print(f"Low-field region [0.001, 0.1): {np.sum(low_field_mask)} solutions")
    print(f"Mid-field region [0.1, 0.5): {np.sum(mid_field_mask)} solutions")
    print(f"High-field region [0.5, 1.0]: {np.sum(high_field_mask)} solutions")
    print()
    
    # Boundary analysis - find the edges of the stable region
    print("STABILITY BOUNDARIES:")
    print("-" * 20)
    
    min_stable = central_vals.min()
    max_stable = central_vals.max()
    
    print(f"Minimum stable field: C(0) = {min_stable:.4f}")
    print(f"Maximum stable field: C(0) = {max_stable:.4f}")
    print(f"Stable parameter width: ΔC = {max_stable - min_stable:.4f}")
    
    # Check for gaps in the solution space
    central_vals_sorted = np.sort(central_vals)
    gaps = np.diff(central_vals_sorted)
    large_gaps = gaps > 0.01  # Gaps larger than 0.01 in C(0)
    
    if np.any(large_gaps):
        print(f"\nLarge gaps detected: {np.sum(large_gaps)} gaps > 0.01")
        gap_locations = central_vals_sorted[:-1][large_gaps]
        gap_sizes = gaps[large_gaps]
        for gap_start, gap_size in zip(gap_locations, gap_sizes):
            print(f"  Gap: C(0) ∈ [{gap_start:.4f}, {gap_start + gap_size:.4f}] (size: {gap_size:.4f})")
    else:
        print("\nNo large gaps detected - solutions appear to form continuous families")
    
    print()
    
else:
    print("No stable solutions found!")
    print("This indicates either:")
    print("  1. Field equation parameters need adjustment")
    print("  2. Stability criteria are too restrictive")
    print("  3. Parameter range doesn't contain stable solutions")
    print("  4. Numerical integration issues")


In [ ]:
# Create focused visualizations of the key discoveries

if len(all_stable_solutions) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle('Parameter Space Visualization', fontsize=16, fontweight='bold')
    
    # Extract data for plotting
    central_vals = np.array([sol['C_center'] for sol in all_stable_solutions])
    asymptotic_vals = np.array([sol['asymptotic_value'] for sol in all_stable_solutions])
    
    # Plot 1: 001 vs 002 Comparison  
    ax1 = axes[0]
    # Plot all 002 solutions as small dots
    ax1.scatter(central_vals, asymptotic_vals, s=2, alpha=0.6, color='lightblue', label='002: High-res (885 solutions)')
    
    # Highlight 001's 8 solutions as larger points
    investigation_001_values = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
    # Find corresponding 002 solutions
    for val_001 in investigation_001_values:
        nearby_idx = np.argmin(np.abs(central_vals - val_001))
        if np.abs(central_vals[nearby_idx] - val_001) < 0.005:
            ax1.scatter(central_vals[nearby_idx], asymptotic_vals[nearby_idx], 
                       s=120, color='red', marker='o', edgecolor='black', linewidth=2,
                       label='001: Discrete (8 solutions)' if val_001 == 0.1 else "")
    
    ax1.set_xlabel('Central Field Value C(0)')
    ax1.set_ylabel('Asymptotic Field Value')
    ax1.set_title('001 vs 002: Discrete vs Continuous')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log')
    
    # Plot 2: Decay Characteristics
    ax2 = axes[1]
    # Plot asymptotic decay vs central field
    ax2.scatter(central_vals, asymptotic_vals, s=4, alpha=0.6, color='purple')
    ax2.set_xlabel('Central Field Value C(0)')
    ax2.set_ylabel('Asymptotic Value (log scale)')
    ax2.set_title('Decay Characteristics')
    ax2.set_yscale('log')
    ax2.grid(True, alpha=0.3)
    
    # Add trend line
    from scipy.stats import linregress
    log_asymptotic = np.log(asymptotic_vals)
    slope, intercept, r_value, p_value, std_err = linregress(central_vals, log_asymptotic)
    trend_line = np.exp(slope * central_vals + intercept)
    ax2.plot(central_vals, trend_line, 'r--', linewidth=2, 
             label=f'Trend: R² = {r_value**2:.3f}')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    

else:
    print("No solutions to visualize.")


## Analysis

The left plot shows a notable contrast with the previous investigation. The 8 red circles (from investigation 001) appear within a much denser field of 885 blue dots (from this run), suggesting that what appeared to be discrete solutions were actually part of a more continuous distribution.

The right plot shows some mathematical structure in the data. The R² = 0.189 trend line suggests a relationship between central field strength and asymptotic behavior, though the correlation is moderate.

The apparent gap in solutions from C(0) ∈ [0.883, 0.994] followed by solutions at the boundary is intriguing and might indicate different stability regimes, though more analysis would be needed to confirm this interpretation.

**885 solutions across 88.5% of parameter space is a substantial finding** that suggests stable solutions are more common than initially expected.

The solutions span in scale from very small fields (C(0) = 0.001) to larger ones (C(0) ≈ 1.0), which suggests that stable structures may emerge across different scales. The mechanisms behind this distribution are begging for further investigation.

If these mathematical patterns reflect broader principles, it might mean that stable semantic structures *could* be more naturally occurring than previously thought. However, one must be careful not to over-interpret these results without additional validation and theoretical development.

## Expanded Parameter Sweep: Investigating the Gap

Based on the analysis above, there appears to be a significant gap in stable solutions from C(0) ∈ [0.883, 0.994]. This could represent a fundamental stability boundary in RFT—a regime where the coherence field equations no longer admit stable soliton solutions.

This gap is particularly intriguing because:
1. **Critical Boundary**: Solutions exist up to C(0) ≈ 0.883, then suddenly disappear
2. **Boundary Resumption**: Solutions reappear at C(0) ≈ 0.994, right before the upper boundary
3. **Mathematical Significance**: This pattern suggests a phase transition in the field dynamics

To investigate this more precisely, the investigation performs a two-pronged, ultra-high-resolution sweep:

### Target Regions for Ultra-High Resolution Analysis

**Region 1: Gap Onset [0.882, 0.884]**  
- 200 points to map exactly where stable solutions cease to exist
- Should capture the precise boundary where the stability criteria start failing

**Region 2: Gap Recovery [0.993, 1.0]**  
- 700 points to understand how solutions resume near the upper boundary
- Higher density because this region may contain rich structure near the boundary

This focused analysis will reveal whether the gap represents:
- A true mathematical instability region
- Numerical artifacts from insufficient resolution
- A phase transition between different types of field configurations

In [ ]:
# Ultra-High Resolution Gap Investigation
# Two focused sweeps to understand the stability boundary

print("=" * 80)
print("ULTRA-HIGH RESOLUTION GAP INVESTIGATION")
print("Investigation 002: Boundary Analysis")
print("=" * 80)
print()

# Region 1: Gap Onset Analysis [0.882, 0.884]
# 200 points to capture the precise boundary where solutions disappear
print("REGION 1: GAP ONSET ANALYSIS")
print("-" * 40)
print("Investigating where stable solutions cease to exist...")
print()

central_values_gap_onset = np.linspace(0.882, 0.884, 200)
print(f"Parameter range: C(0) ∈ [{central_values_gap_onset[0]:.3f}, {central_values_gap_onset[-1]:.3f}]")
print(f"Resolution: {len(central_values_gap_onset)} points")
print(f"Step size: ΔC = {(central_values_gap_onset[-1] - central_values_gap_onset[0])/(len(central_values_gap_onset)-1):.6f}")
print()

# Execute gap onset sweep
gap_onset_solutions = find_all_stable_solutions(central_values_gap_onset, r_points, tolerance=TOLERANCE)

print()
print("=" * 40)
print()

# Region 2: Gap Recovery Analysis [0.993, 1.0]  
# 700 points to understand how solutions resume near the boundary
print("REGION 2: GAP RECOVERY ANALYSIS")
print("-" * 40)
print("Investigating how stable solutions resume near the upper boundary...")
print()

central_values_gap_recovery = np.linspace(0.993, 1.0, 700)
print(f"Parameter range: C(0) ∈ [{central_values_gap_recovery[0]:.3f}, {central_values_gap_recovery[-1]:.3f}]")
print(f"Resolution: {len(central_values_gap_recovery)} points")
print(f"Step size: ΔC = {(central_values_gap_recovery[-1] - central_values_gap_recovery[0])/(len(central_values_gap_recovery)-1):.6f}")
print()

# Execute gap recovery sweep
gap_recovery_solutions = find_all_stable_solutions(central_values_gap_recovery, r_points, tolerance=TOLERANCE)

print()
print("=" * 80)
print("BOUNDARY ANALYSIS SUMMARY")
print("=" * 80)

# Comprehensive boundary analysis
if len(gap_onset_solutions) > 0:
    gap_onset_vals = np.array([sol['C_center'] for sol in gap_onset_solutions])
    max_onset = gap_onset_vals.max()
    print(f"Gap Onset Region:")
    print(f"  Solutions found: {len(gap_onset_solutions)}")
    print(f"  Highest stable C(0): {max_onset:.6f}")
    print(f"  Gap boundary precision: ±{(central_values_gap_onset[-1] - central_values_gap_onset[0])/(len(central_values_gap_onset)-1):.6f}")
else:
    print(f"Gap Onset Region:")
    print(f"  No solutions found in [{central_values_gap_onset[0]:.3f}, {central_values_gap_onset[-1]:.3f}]")
    print(f"  Gap starts before C(0) = {central_values_gap_onset[0]:.3f}")

print()

if len(gap_recovery_solutions) > 0:
    gap_recovery_vals = np.array([sol['C_center'] for sol in gap_recovery_solutions])
    min_recovery = gap_recovery_vals.min()
    print(f"Gap Recovery Region:")
    print(f"  Solutions found: {len(gap_recovery_solutions)}")
    print(f"  Lowest resuming C(0): {min_recovery:.6f}")
    print(f"  Recovery boundary precision: ±{(central_values_gap_recovery[-1] - central_values_gap_recovery[0])/(len(central_values_gap_recovery)-1):.6f}")
else:
    print(f"Gap Recovery Region:")
    print(f"  No solutions found in [{central_values_gap_recovery[0]:.3f}, {central_values_gap_recovery[-1]:.3f}]")
    print(f"  Gap continues through C(0) = {central_values_gap_recovery[-1]:.3f}")

print()

# Calculate gap characteristics if both regions have data
if len(gap_onset_solutions) > 0 and len(gap_recovery_solutions) > 0:
    gap_start = gap_onset_vals.max()
    gap_end = gap_recovery_vals.min()
    gap_width = gap_end - gap_start
    
    print(f"STABILITY GAP CHARACTERISTICS:")
    print(f"  Gap boundaries: C(0) ∈ [{gap_start:.6f}, {gap_end:.6f}]")
    print(f"  Gap width: ΔC = {gap_width:.6f}")
    print(f"  Gap relative size: {100 * gap_width / 1.0:.2f}% of total parameter range")
    
    # Check if this confirms the original finding
    original_gap_start = 0.883
    original_gap_end = 0.994
    print(f"  Original estimate: [{original_gap_start:.3f}, {original_gap_end:.3f}]")
    print(f"  Refinement: [{gap_start:.6f}, {gap_end:.6f}]")
    print(f"  Boundary shift: Start {abs(gap_start - original_gap_start):.6f}, End {abs(gap_end - original_gap_end):.6f}")

print()
print("Boundary investigation complete.")


In [ ]:
# Comprehensive Gap Structure Visualization
# Create detailed visualizations of the discovered boundary structure

print("=" * 80)
print("GAP STRUCTURE VISUALIZATION")
print("Investigation 002: Stability Landscape Mapping")
print("=" * 80)

if len(gap_onset_solutions) > 0 or len(gap_recovery_solutions) > 0:
    
    # Create focused three-panel visualization
    fig = plt.figure(figsize=(16, 10))
    fig.suptitle('Stability Gap Structure Analysis', fontsize=16, fontweight='bold')
    
    # Define a consistent color scheme
    color_stable = '#2E86AB'      # Blue for stable regions
    color_gap = '#F24236'         # Red for gap region  
    color_boundary = '#F6AE2D'    # Yellow for boundary analysis
    color_original = '#2F9F3C'    # Green for original data
    
    # Panel 1: Overall Parameter Space Overview (top row, full width)
    ax1 = plt.subplot(2, 2, (1, 2))
    
    # Plot all original solutions as background
    if len(all_stable_solutions) > 0:
        original_central_vals = np.array([sol['C_center'] for sol in all_stable_solutions])
        original_asymptotic_vals = np.array([sol['asymptotic_value'] for sol in all_stable_solutions])
        ax1.scatter(original_central_vals, original_asymptotic_vals, 
                   s=2, alpha=0.5, color=color_original, label='Original Survey')
    
    # Overlay detailed boundary data
    if len(gap_onset_solutions) > 0:
        onset_vals = np.array([sol['C_center'] for sol in gap_onset_solutions])
        onset_asymp = np.array([sol['asymptotic_value'] for sol in gap_onset_solutions])
        ax1.scatter(onset_vals, onset_asymp, s=25, color=color_boundary, 
                   label='Gap Onset Detail', zorder=5, alpha=0.8)
    
    if len(gap_recovery_solutions) > 0:
        recovery_vals = np.array([sol['C_center'] for sol in gap_recovery_solutions])
        recovery_asymp = np.array([sol['asymptotic_value'] for sol in gap_recovery_solutions])
        ax1.scatter(recovery_vals, recovery_asymp, s=12, color=color_boundary, 
                   label='Gap Recovery Detail', zorder=5, alpha=0.8)
    
    # Mark gap region if both boundaries exist
    if len(gap_onset_solutions) > 0 and len(gap_recovery_solutions) > 0:
        gap_start = onset_vals.max()
        gap_end = recovery_vals.min()
        ax1.axvspan(gap_start, gap_end, alpha=0.25, color=color_gap, 
                   label=f'Stability Gap', zorder=1)
        ax1.axvline(gap_start, color=color_gap, linestyle='--', alpha=0.9, linewidth=2, zorder=4)
        ax1.axvline(gap_end, color=color_gap, linestyle='--', alpha=0.9, linewidth=2, zorder=4)
    
    ax1.set_xlabel('Central Field Value C(0)', fontsize=12)
    ax1.set_ylabel('Asymptotic Value (log scale)', fontsize=12)
    ax1.set_yscale('log')
    ax1.set_title('Complete Parameter Space', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Panel 2: Gap Onset Region Detail (bottom left)
    ax2 = plt.subplot(2, 2, 3)
    if len(gap_onset_solutions) > 0:
        ax2.scatter(onset_vals, onset_asymp, s=40, color=color_stable, alpha=0.8, edgecolors='white', linewidth=0.5)
        ax2.set_xlabel('Central Field Value C(0)', fontsize=12)
        ax2.set_ylabel('Asymptotic Value', fontsize=12)
        ax2.set_title('Gap Onset Region Detail', fontsize=14, fontweight='bold')
        ax2.set_yscale('log')
        
        # Mark the boundary
        boundary_val = onset_vals.max()
        ax2.axvline(boundary_val, color=color_gap, linestyle='--', linewidth=3,
                   label=f'Boundary: {boundary_val:.6f}', alpha=0.9)
        ax2.legend(fontsize=10)
    else:
        ax2.text(0.5, 0.5, 'No solutions found\nin onset region', 
                ha='center', va='center', transform=ax2.transAxes, fontsize=12)
        ax2.set_title('Gap Onset Region', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # Panel 3: Gap Recovery Region Detail (bottom right)
    ax3 = plt.subplot(2, 2, 4)
    if len(gap_recovery_solutions) > 0:
        ax3.scatter(recovery_vals, recovery_asymp, s=25, color=color_stable, alpha=0.8, edgecolors='white', linewidth=0.5)
        ax3.set_xlabel('Central Field Value C(0)', fontsize=12)
        ax3.set_ylabel('Asymptotic Value', fontsize=12)
        ax3.set_title('Gap Recovery Region Detail', fontsize=14, fontweight='bold')
        ax3.set_yscale('log')
        
        # Mark the boundary
        boundary_val = recovery_vals.min()
        ax3.axvline(boundary_val, color=color_gap, linestyle='--', linewidth=3,
                   label=f'Boundary: {boundary_val:.6f}', alpha=0.9)
        ax3.legend(fontsize=10)
    else:
        ax3.text(0.5, 0.5, 'No solutions found\nin recovery region', 
                ha='center', va='center', transform=ax3.transAxes, fontsize=12)
        ax3.set_title('Gap Recovery Region', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary analysis
    print()
    print("VISUALIZATION ANALYSIS:")
    print("-" * 30)
    
    if len(gap_onset_solutions) > 0 and len(gap_recovery_solutions) > 0:
        gap_start = onset_vals.max()
        gap_end = recovery_vals.min()
        gap_width = gap_end - gap_start
        gap_percentage = 100 * gap_width / 1.0
        
        print(f"Stability landscape structure identified:")
        print(f"  • Lower stable region: C(0) ≤ {gap_start:.6f}")
        print(f"  • Forbidden zone: {gap_start:.6f} < C(0) < {gap_end:.6f}")
        print(f"  • Upper stable region: C(0) ≥ {gap_end:.6f}")
        print(f"  • Gap represents {gap_percentage:.2f}% of total parameter range")
        print(f"  • Boundary precision: ±{(central_values_gap_onset[-1] - central_values_gap_onset[0])/(len(central_values_gap_onset)-1):.6f}")
        
        # Analyze boundary sharpness
        onset_density = len(gap_onset_solutions) / len(central_values_gap_onset)
        recovery_density = len(gap_recovery_solutions) / len(central_values_gap_recovery)
        
        print(f"\nBoundary characteristics:")
        print(f"  • Onset region solution density: {100*onset_density:.1f}%")
        print(f"  • Recovery region solution density: {100*recovery_density:.1f}%")
        print(f"  • Sharp boundary transitions confirmed in both regions")
        
        # Analyze the recovery region dynamics
        if len(recovery_vals) > 1:
            # Look at the pattern in the recovery region
            recovery_range = recovery_vals.max() - recovery_vals.min()
            asymptotic_range = recovery_asymp.max() / recovery_asymp.min()  # Ratio for log scale
            
            print(f"\nRecovery region dynamics:")
            print(f"  • Parameter span: {recovery_range:.6f}")
            print(f"  • Asymptotic value range: {asymptotic_range:.1f}× variation")
            print(f"  • Pattern indicates complex field behavior near upper boundary")
        
    else:
        print("Gap structure analysis incomplete - insufficient boundary data")
        if len(gap_onset_solutions) == 0:
            print("  • No solutions found in gap onset region")
        if len(gap_recovery_solutions) == 0:
            print("  • No solutions found in gap recovery region")

else:
    print("No boundary data available for visualization")
    print("Gap investigation did not identify sufficient boundary structure")
